In [1]:
import numpy as np
import operator
import pandas as pd
import random

from pylsl import StreamInfo, StreamOutlet
import numpy as np
import pandas as pd
import time
from psychopy import visual, core, event
from glob import glob

file_path = "5_levels_questions.csv"
df_csv_questions = pd.read_csv(file_path, delimiter=",")

# Replace Nan to 0
df_csv_questions.fillna(value=0, inplace=True)

def generate_question_list(df):
    list_indexs = []
    start_index = 0
    end_index = 0

    for ii in range(5):

        indexs = random.sample(range(0+start_index,9+end_index), 2)
        list_indexs.append(indexs[0])
        list_indexs.append(indexs[1])
        start_index += 10
        end_index += 10

    df_selected_question = df.iloc[list_indexs, :]
    #print(df_selected_question)
    
    return df_selected_question

def generate_level_question(df,difficulty_level, number):
    
    df_level_equations = df.loc[df["Difficulty level"] == difficulty_level, ["A", "B", "C", "D", "E", "F"]]
    list_level_eq = df.loc[df["Difficulty level"] == difficulty_level, ["Equation"]].values
    list_level_equ_values = df_level_equations.values

    # first number of level one
    #print("The first index in 1st difficulty is {}".format(np.min(df_level_equations.index)))

    # last number of level one
    #print("The first index in 1st difficulty is {}".format(np.max(df_level_equations.index)))

    # Size of question in 1st Difficulty level
    #print("The total equations in 1st difficulty is {}".format(df_level_equations.shape[0]))

    # Each level has len(list_level_equ_values) questions
    random_number = number
    
    Equation = list_level_eq[random_number]
    #print(Equation)
    
    selected_equation_number = list_level_equ_values[random_number]
    #print(selected_equation_number)

    if difficulty_level == 1:
        Total = selected_equation_number[0] + selected_equation_number[1] + selected_equation_number[2] + selected_equation_number[3] + selected_equation_number[4]
    elif difficulty_level == 2:
        Total = selected_equation_number[0] + selected_equation_number[1] - selected_equation_number[2] + selected_equation_number[3] - selected_equation_number[4]
    elif difficulty_level == 3:
        Total = (selected_equation_number[0] * selected_equation_number[1]) + selected_equation_number[2] - selected_equation_number[3] + selected_equation_number[4]
    elif difficulty_level == 4:
        Total = (selected_equation_number[0] / selected_equation_number[1]) + selected_equation_number[2] - (selected_equation_number[3] * selected_equation_number[4])
    elif difficulty_level == 5:
        Total = selected_equation_number[0] + (selected_equation_number[1] * selected_equation_number[2]) + (selected_equation_number[3] / selected_equation_number[4]) - selected_equation_number[5]

    #print("Final Total: {}".format(Total))

    Correct_answer = np.int(Total)
    
    if len(str(np.int(Correct_answer))) == 1:
        Wrong_choice_01 = np.random.randint(low=0, high=10, size=1)[0] + Correct_answer
        Wrong_choice_02 = np.random.randint(low=0, high=10, size=1)[0] + Correct_answer + 1
    
    elif len(str(np.int(Correct_answer))) == 2:
        Wrong_choice_01 = np.random.randint(low=10, high=100, size=1)[0] + Correct_answer
        Wrong_choice_02 = np.random.randint(low=10, high=100, size=1)[0] + Correct_answer + 1
        
    else:
        Wrong_choice_01 = np.random.randint(low=100, high=1000, size=1)[0] + Correct_answer
        Wrong_choice_02 = np.random.randint(low=100, high=1000, size=1)[0] + Correct_answer + 1

    #print("Correct answer: {}".format(Correct_answer))
    #print("Wrong answer: {} and {}".format(Wrong_choice_01, Wrong_choice_02))
    
    return [Equation, Correct_answer, Wrong_choice_01, Wrong_choice_02]


def display_intro():
    title = "** Easy Math Quiz **"
    print("*" * len(title))
    print(title)
    print("*" * len(title))

def display_separator():
    print("-" * 24)
    
def get_user_input():

    # bool("") is False
    user_input = int(input("Enter your choice: ") or "9999")

    while user_input > 3 or user_input <= 0 :
        print("Invalid choice option.")
        user_input = int(input("Please try again: ") or "9999")
    else:
        return user_input
    
def display_question(question, Correct_score):
    
    # Extract value of Correct_score outside of this function
    
    print("Question: {}".format(question[0]))
    print("Choices")
    
    # Shuffle choices
    #np_arr = np.arange(1,4)
    #np.random.shuffle(np_arr)
    
    # Where is the Correct answre??? (I do not know either!!)
    print("1) {}".format(question[np_arr[0]]))
    print("2) {}".format(question[np_arr[1]]))
    print("3) {}".format(question[np_arr[2]]))
    
    Correct_answer = question[1] # Extract the Correct answer
    #print(Correct_answer)
    selected_choice = get_user_input()
    #print(question[np_arr[selected_choice-1]])
    
    if Correct_answer == question[np_arr[selected_choice-1]]:
        print("That is correct!!")
        print("Your choice is {}".format(question[np_arr[selected_choice-1]]))
        Correct_score += 1
    else:
        print("That is wrong. TOO BAD!!")
        print("The correct answer is {}".format(Correct_answer))
    
    return Correct_score

def display_question_without_user_input(question):
    
    # Extract value of Correct_score outside of this function
    
    print("Question: {}".format(question[0]))
    print("Choices")
    
    # Shuffle choices
    #np_arr = np.arange(1,4)
    #np.random.shuffle(np_arr)
    
    # Where is the Correct answre??? (I do not know either!!)
    print("1) {}".format(question[np_arr[0]]))
    print("2) {}".format(question[np_arr[1]]))
    print("3) {}".format(question[np_arr[2]]))
    
    
    
    


In [2]:
#setup parameters
#this requires expertise and paper reading to know what are the appropriate parameters
n_trials = 10       # 10 questions per block 
rest_30  = 30       # 30 Rest time between block
iti      = 10        # 5  inter trial interval, i.e., rest duration
soa      = 40        # 40 Stimulus-onset asynchrony, i.e., how long the stimulus will stay
record_duration = np.float32(99999) # 10000 = 10,000 ms ??

position = np.ones(shape=10)
trials = pd.DataFrame(dict(position=position, timestamp=np.zeros(n_trials)))


#name, type, channel_count, sampling rate, channel format, source_id
#Note that Markers, 1, and 0.0 cannot be altered
#info = StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'int32', 'CytonMarkerID')#make an outlet
info = StreamInfo('CytonMarkers', 'Markers', 1, 0.0, 'string', 'CytonMarkerID')#make an outlet
outlet = StreamOutlet(info)


textSize = 30

In [ ]:
#mywin = visual.Window([1200, 700], fullscr=False)
#mywin = visual.Window([1200,700], color="black", fullscr=False,allowGUI=False, winType='pyglet',monitor="testMonitor", units="pix") #open a window
mywin = visual.Window([1200,700], fullscr=False, winType='pyglet',monitor="testMonitor", units="pix") #open a window
block = 0

while True:
    roundEX = str(block+1)
    message1 = visual.TextStim(mywin, text='Round :'+roundEX, pos = (0,200), height=40)
    message1.draw()
    message = visual.TextStim(mywin, text='Solve the following math equation , press space to start' , height=textSize , pos = (0,-100))
    message.draw()
    mywin.flip()
    
    
    
    while block < 3:
        
        keys = event.getKeys()
        # Selected 10 questions (2 questions from each level)
        df_selected_questions = generate_question_list(df_csv_questions)

        # Each question_xx contains [Equation, Correct_answer, Wrong_choice_01, Wrong_choice_02]
        question_01 = generate_level_question(df_selected_questions,  difficulty_level=1, number=0)
        question_02 = generate_level_question(df_selected_questions,  difficulty_level=1, number=1)
        question_03 = generate_level_question(df_selected_questions,  difficulty_level=2, number=0)
        question_04 = generate_level_question(df_selected_questions,  difficulty_level=2, number=1)
        question_05 = generate_level_question(df_selected_questions,  difficulty_level=3, number=0)
        question_06 = generate_level_question(df_selected_questions,  difficulty_level=3, number=1)
        question_07 = generate_level_question(df_selected_questions,  difficulty_level=4, number=0)
        question_08 = generate_level_question(df_selected_questions,  difficulty_level=4, number=1)
        question_09 = generate_level_question(df_selected_questions,  difficulty_level=5, number=0)
        question_10 = generate_level_question(df_selected_questions,  difficulty_level=5, number=1)
        list_questions = [question_01, question_02, question_03, question_04, question_05, 
                          question_06, question_07, question_08, question_09, question_10]

        if 'space' in keys:  # If space has been pushed
            message.setText = '' # Clear the screen
            message.draw()
            mywin.flip()

            display_intro()
            display_separator()

            n_question = 0
            Correct_score = 0

            # Loop for trail
            for ii, trial in trials.iterrows():

                # onset
                pos = trials['position'].iloc[ii]  #running each position, using index as iterator (.iloc)

                # Rrandom positions of choices
                np_arr = np.arange(1,4)
                np.random.shuffle(np_arr)

                lEndPoint = -(mywin.size[0]/2)
                rEndPoint = (mywin.size[0]/2)
                line = visual.ShapeStim(mywin, vertices=[(lEndPoint, 0), (rEndPoint, 0)],lineColor = "blue", lineWidth = 30, pos = (0,300))
                dist = mywin.size[0]/(90*soa)
                timer = core.CountdownTimer(soa)
                
                
                
                # sent the start marker when the question start "block,trail"
                markerString = str(block+1)+","+str(ii+1)
                print("Start mark is {}".format(markerString))
                outlet.push_sample([markerString])
                
                display_question_without_user_input(list_questions[ii])
                
                ## this loop use to display moving bar
                while timer.getTime() > 0:
                    
                    rEndPoint -= dist
                    line.vertices = [(lEndPoint,0),(rEndPoint,0)]
                    line.draw()
                    message_equation = visual.TextStim(mywin, text=str(list_questions[ii][0][0]), height=textSize,  pos = (0,0)) # get a equation of question
                    message_choice_01 = visual.TextStim(mywin, text= "1) " + str(list_questions[ii][np_arr[0]]), height=textSize, pos = (-200,-200))  
                    message_choice_02 = visual.TextStim(mywin, text= "2) " + str(list_questions[ii][np_arr[1]]), height=textSize, pos = (0,-200)) 
                    message_choice_03 = visual.TextStim(mywin, text= "3) " + str(list_questions[ii][np_arr[2]]), height=textSize,  pos = (200,-200))  
                    message_equation.draw()
                    message_choice_01.draw()
                    message_choice_02.draw()
                    message_choice_03.draw()
                    mywin.flip()
                    n_question += 1
                    
                # sent the end of trail marker = "-block,-trails"
                markerString = str(-(block+1))+","+str(-(ii+1))
                print("End mark is {}".format(markerString))
                outlet.push_sample([markerString])
                display_separator()
                
                message = visual.TextStim(mywin, text='Answer you answer,    RELAX for 10 second')
                message.draw()
                mywin.flip() 

                # inter trial interval
                core.wait(iti)
                mywin.flip() #clear fixation 
                
            block +=1
            if block != 3:
                message = visual.TextStim(mywin, text='Rest for 30 seconds')
                message.draw()
                mywin.flip() #refresh
            
            if block == 3:
                message = visual.TextStim(mywin, text='FINISH!', height=textSize )
                message.draw()
                mywin.flip() #refresh
                core.wait(2)
                mywin.close()
                
            # Rest between block    
            core.wait(rest_30) # rest 30 seconds
            
            break;
            
    if block == 3:
        break

mywin.close()  #do not delete, otherwise, the window will not turn off

********************
** Easy Math Quiz **
********************
------------------------
Start mark is 1,1
Question: ['35 + 68 + 35 + 65 + 46']
Choices
1) 249
2) 1040
3) 1016


In [4]:
mywin.close()